In [1]:
# %pip install -r ../requirements.txt

In [2]:
# CUDA 12.1 DOWNLOAD
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [3]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
import dill
import joblib

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

In [5]:
os.environ['TRANSFORMERS_CACHE'] = './models/cache'
os.environ['HF_HOME'] = './models/cache'

In [6]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, cohen_kappa_score, r2_score
from sentence_transformers import SentenceTransformer

e:\SEMESTER 6\NLP\UAS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\SEMESTER 6\NLP\UAS\.venv\Lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [7]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import optuna
import language_tool_python

In [8]:
from textstat import flesch_kincaid_grade
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import RobertaTokenizer, RobertaModel

In [10]:
#comment if do not want to use cuda
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
0
NVIDIA GeForce RTX 3050 Laptop GPU


In [11]:
nltk.data.path.append(os.path.abspath('nltk_data'))

In [12]:

# nltk.download('punkt_tab', download_dir='nltk_data')
# nltk.download('omw-1.4', download_dir='nltk_data')
# nltk.download('stopwords' , download_dir='nltk_data')
# nltk.download('wordnet' , download_dir='nltk_data')
# nltk.download('averaged_perceptron_tagger_eng' , download_dir='nltk_data')

---


In [13]:
dataset_path = os.path.abspath('../Dataset/dataset mamasmasmudi.xlsx')

In [14]:
def load_data(dataset_path):
    df_dpk_tkj = pd.read_excel(dataset_path, sheet_name='DPK (TKJ)')
    df_mpp_rpl = pd.read_excel(dataset_path, sheet_name='MPP (RPL)')
    df_mpp_ppl = pd.read_excel(dataset_path, sheet_name='MPP (PPL) 2')
    df_mpp_tkj = pd.read_excel(dataset_path, sheet_name='MPP (TKJ-Telkom)')
    df_kunci_jawaban = pd.read_excel(dataset_path, sheet_name='Kunci Jawaban')
    
    df = pd.concat([df_dpk_tkj, df_mpp_rpl, df_mpp_ppl, df_mpp_tkj])
    
    return df, df_kunci_jawaban

In [15]:
essays, jawaban_essay = load_data(dataset_path)

In [16]:
essays

,Kelas,Jurusan,Mapel,Nama,Pertanyaan,Kode,Jawaban,Nilai
0,X,TKJ,DPK,Zainur Ridho,Sebutkan 1 bisnis komputer yang anda ingin ban...,1,Berikut adalah contoh bisnis di bidang teknik ...,1.0
1,X,TKJ,DPK,Zainur Ridho,Sebutkan 5 dampak negatif perkembangan teknolo...,2,Pencemaran limbah dan bahan berbahaya\nMeningk...,2.0
2,X,TKJ,DPK,Zainur Ridho,Sebutkan 3 ciri-ciri dari jaringan seluler gen...,3,3G-Digital broadband packet data (CDMA EV-DO U...,2.0
3,X,TKJ,DPK,Zainur Ridho,Sebutkan dan jelaskan 3 hardware komputer yang...,4,Fungsi hardware menerima output\nFungsi hardwa...,2.3
4,X,TKJ,DPK,Uswatun Hasanah,Sebutkan 1 bisnis komputer yang anda ingin ban...,1,Jual beli komputer dan aksesorisnya yaitu menj...,3.0
...,...,...,...,...,...,...,...,...
159,XI,TKJ,MPP,Agus Arifin,"Jelaskan fungsi dan sensor LDR, Infrared, dan ...",10,LDR digunakan untuk kontrol lampu di taman\nIn...,2.5
160,XI,TKJ,MPP,Agus Arifin,Jelaskan Jenis Arduino yang paling mudah dipel...,11,Arduino uno karena banyak sekali reserensi yan...,2.0
161,XI,TKJ,MPP,Agus Arifin,"Jelaskan fungsi dari kabel jumper, breadboard ...",12,Kabel jumper = sbg konduktor listrik\nbreadboa...,3.0
162,XI,TKJ,MPP,Agus Arifin,Proyek sederhana yang dapat dimanfaatkan dalam...,13,Colokan listrik pintar\nPenerapan IOT pada tem...,2.5


In [17]:
jawaban_essay

,Kode,jurusan,Kelas,Mata Pelajaran,Pertanyaan,Jawaban
0,1,TKJ,X,DPK,Sebutkan 1 bisnis komputer yang anda ingin ban...,Bisnis Komputer yang Ingin Saya Bangun:\nBisni...
1,2,TKJ,X,DPK,Sebutkan 5 dampak negatif perkembangan teknolo...,5 Dampak Negatif Perkembangan Teknologi:\nKete...
2,3,TKJ,X,DPK,Sebutkan 3 ciri-ciri dari jaringan seluler gen...,3 Ciri-ciri Jaringan Seluler Generasi 3 (3G):\...
3,4,TKJ,X,DPK,Sebutkan dan jelaskan 3 hardware komputer yang...,3 Hardware Komputer dan Penjelasan:\nCPU (Cent...
4,5,RPL,XI,MPP,Jelaskan ancaman keamanan yang mungkin terjadi...,Ancaman Keamanan Mengunduh Driver Perangkat Ke...
5,6,RPL,XI,MPP,Jelaskan proses langkah-langkah menyalakan dan...,Menyalakan Komputer:\n\nPastikan semua kabel p...
6,7,RPL,XI,MPP,Jelaskan pengertian sistem operasi (SO) dan se...,Sistem operasi (SO) adalah perangkat lunak sis...
7,8,RPL,XI,MPP,Bagaimana perbedaan antara perangkat keras (ha...,Perangkat keras (hardware) adalah komponen fis...
8,9,RPL,XI,MPP,Jelaskan perbedaan utama antara bus internal d...,Bus internal:\nJalur komunikasi yang menghubun...
9,10,TKJ,XI,MPP,"Jelaskan fungsi dan sensor LDR, Infrared, dan ...",1. LDR (Light Dependent Resistor):\nFungsi: Se...


In [18]:
essays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 519 entries, 0 to 163
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Kelas       519 non-null    object 
 1   Jurusan     519 non-null    object 
 2   Mapel       519 non-null    object 
 3   Nama        519 non-null    object 
 4   Pertanyaan  519 non-null    object 
 5   Kode        519 non-null    int64  
 6   Jawaban     519 non-null    object 
 7   Nilai       519 non-null    float64
dtypes: float64(1), int64(1), object(6)
memory usage: 36.5+ KB


In [19]:
jawaban_essay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Kode            14 non-null     int64 
 1   jurusan         14 non-null     object
 2   Kelas           14 non-null     object
 3   Mata Pelajaran  14 non-null     object
 4   Pertanyaan      14 non-null     object
 5   Jawaban         14 non-null     object
dtypes: int64(1), object(5)
memory usage: 804.0+ bytes


In [20]:
essays.describe()

,Kode,Nilai
count,519.000000,519.000000
mean,7.815029,2.769942
std,3.533142,0.902924
min,1.000000,0.000000
25%,5.000000,2.500000
50%,8.000000,3.000000
75%,11.000000,3.000000
max,14.000000,4.000000


In [21]:
essays['Nilai'].unique()

array([1. , 2. , 2.3, 3. , 4. , 2.5, 2.2, 2.1, 3.1, 0.1, 3.5, 1.5, 2.7,
       3.8, 3.2, 2.8, 2.9, 0. ])

---


# Preprocess


In [ ]:

class EssayPreprocessor:
    def __init__(self):
        self.stop_words = set(stopwords.words('indonesian'))
        self.sbert = SentenceTransformer('firqaaa/indo-sentence-bert-base',
                                            device="cuda" if torch.cuda.is_available() else "cpu")

    def preprocess_text(self, text):
        if not text:
            return ""
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s]', '', text)
        # text = re.sub(r'\d+', '', text) 

        tokens = word_tokenize(text)
        tokens = [t for t in tokens if t not in self.stop_words]
        return ' '.join(tokens)

    def extract_linguistic_features(self, text):
        if not text:
            return {}

        words = word_tokenize(text)
        unique_words = set(words)
        vocab_richness = len(unique_words) / max(1, len(words))

        sentences = sent_tokenize(text)
        avg_sentence_len = sum(len(word_tokenize(s)) for s in sentences) / max(1, len(sentences))


        return {
            'vocab_richness': vocab_richness,
            'avg_sentence_len': avg_sentence_len,
            'total_words': len(words),

        }

    def get_sbert_embedding(self, text):
        return self.sbert.encode(text, show_progress_bar=False)

    def __getstate__(self):
        state = self.__dict__.copy()
        del state['sbert']
        return state

    def __setstate__(self, state):
        self.__dict__.update(state)
        self.sbert = SentenceTransformer('firqaaa/indo-sentence-bert-base',
                                            device="cuda" if torch.cuda.is_available() else "cpu")


In [23]:
preprocessor = EssayPreprocessor()

In [24]:
essays['processed_text'] = essays['Jawaban'].apply(preprocessor.preprocess_text)
jawaban_essay['processed_kunci_jawaban'] = jawaban_essay['Jawaban'].apply(preprocessor.preprocess_text)



In [25]:
essays

,Kelas,Jurusan,Mapel,Nama,Pertanyaan,Kode,Jawaban,Nilai,processed_text
0,X,TKJ,DPK,Zainur Ridho,Sebutkan 1 bisnis komputer yang anda ingin ban...,1,Berikut adalah contoh bisnis di bidang teknik ...,1.0,contoh bisnis bidang teknik komputer jaringan ...
1,X,TKJ,DPK,Zainur Ridho,Sebutkan 5 dampak negatif perkembangan teknolo...,2,Pencemaran limbah dan bahan berbahaya\nMeningk...,2.0,pencemaran limbah bahan berbahaya meningkatnya...
2,X,TKJ,DPK,Zainur Ridho,Sebutkan 3 ciri-ciri dari jaringan seluler gen...,3,3G-Digital broadband packet data (CDMA EV-DO U...,2.0,3gdigital broadband packet data cdma evdo umts...
3,X,TKJ,DPK,Zainur Ridho,Sebutkan dan jelaskan 3 hardware komputer yang...,4,Fungsi hardware menerima output\nFungsi hardwa...,2.3,fungsi hardware menerima output fungsi hardwar...
4,X,TKJ,DPK,Uswatun Hasanah,Sebutkan 1 bisnis komputer yang anda ingin ban...,1,Jual beli komputer dan aksesorisnya yaitu menj...,3.0,jual beli komputer aksesorisnya menjual laptop...
...,...,...,...,...,...,...,...,...,...
159,XI,TKJ,MPP,Agus Arifin,"Jelaskan fungsi dan sensor LDR, Infrared, dan ...",10,LDR digunakan untuk kontrol lampu di taman\nIn...,2.5,ldr kontrol lampu taman infrared penciptaan pa...
160,XI,TKJ,MPP,Agus Arifin,Jelaskan Jenis Arduino yang paling mudah dipel...,11,Arduino uno karena banyak sekali reserensi yan...,2.0,arduino uno reserensi membahas arduino uno
161,XI,TKJ,MPP,Agus Arifin,"Jelaskan fungsi dari kabel jumper, breadboard ...",12,Kabel jumper = sbg konduktor listrik\nbreadboa...,3.0,kabel jumper sbg konduktor listrik breadboard ...
162,XI,TKJ,MPP,Agus Arifin,Proyek sederhana yang dapat dimanfaatkan dalam...,13,Colokan listrik pintar\nPenerapan IOT pada tem...,2.5,colokan listrik pintar penerapan iot sampah al...


In [26]:
jawaban_essay

,Kode,jurusan,Kelas,Mata Pelajaran,Pertanyaan,Jawaban,processed_kunci_jawaban
0,1,TKJ,X,DPK,Sebutkan 1 bisnis komputer yang anda ingin ban...,Bisnis Komputer yang Ingin Saya Bangun:\nBisni...,bisnis komputer bangun bisnis jasa pembuatan p...
1,2,TKJ,X,DPK,Sebutkan 5 dampak negatif perkembangan teknolo...,5 Dampak Negatif Perkembangan Teknologi:\nKete...,5 dampak negatif perkembangan teknologi keterg...
2,3,TKJ,X,DPK,Sebutkan 3 ciri-ciri dari jaringan seluler gen...,3 Ciri-ciri Jaringan Seluler Generasi 3 (3G):\...,3 ciriciri jaringan seluler generasi 3 3g kece...
3,4,TKJ,X,DPK,Sebutkan dan jelaskan 3 hardware komputer yang...,3 Hardware Komputer dan Penjelasan:\nCPU (Cent...,3 hardware komputer penjelasan cpu central pro...
4,5,RPL,XI,MPP,Jelaskan ancaman keamanan yang mungkin terjadi...,Ancaman Keamanan Mengunduh Driver Perangkat Ke...,ancaman keamanan mengunduh driver perangkat ke...
5,6,RPL,XI,MPP,Jelaskan proses langkah-langkah menyalakan dan...,Menyalakan Komputer:\n\nPastikan semua kabel p...,menyalakan komputer pastikan kabel power terpa...
6,7,RPL,XI,MPP,Jelaskan pengertian sistem operasi (SO) dan se...,Sistem operasi (SO) adalah perangkat lunak sis...,sistem operasi so perangkat lunak sistem menge...
7,8,RPL,XI,MPP,Bagaimana perbedaan antara perangkat keras (ha...,Perangkat keras (hardware) adalah komponen fis...,perangkat keras hardware komponen fisik komput...
8,9,RPL,XI,MPP,Jelaskan perbedaan utama antara bus internal d...,Bus internal:\nJalur komunikasi yang menghubun...,bus internal jalur komunikasi menghubungkan ko...
9,10,TKJ,XI,MPP,"Jelaskan fungsi dan sensor LDR, Infrared, dan ...",1. LDR (Light Dependent Resistor):\nFungsi: Se...,1 ldr light dependent resistor fungsi sensor c...


In [ ]:
linguistic_features = essays['processed_text'].apply(preprocessor.extract_linguistic_features)
linguistic_df =  pd.json_normalize(linguistic_features)

In [ ]:
linguistic_df

In [ ]:
linguistic_df.describe()

In [ ]:
print("Generating SBERT embeddings...")
sbert_embeddings = np.array(essays['processed_text'].apply(preprocessor.get_sbert_embedding).tolist())
sbert_df = pd.DataFrame(sbert_embeddings)

In [ ]:
sbert_df

In [ ]:
X = pd.concat([linguistic_df, sbert_df], axis=1)
y = essays['Nilai']

In [ ]:
X

In [ ]:
y

---


In [ ]:
class AESModel(nn.Module):
    def __init__(self, input_dim):
        super(AESModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.drop1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.drop2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, 64)
        self.fc_out = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.drop2(x)
        x = self.fc3(x)
        x = self.relu(x)
        return self.fc_out(x)

In [ ]:
def train_model(X_train, y_train, params):

    X_train_tensor = torch.FloatTensor(X_train.values)
    y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1)
    
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)

    model = AESModel(input_dim=X_train.shape[1])
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
    

    for epoch in range(params['epochs']):
        model.train()
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
    
    return model

In [ ]:

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test.values)
        predictions = model(X_test_tensor).squeeze().numpy()

    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)
    
    return {
        'predictions': predictions,
        'mse': mse,
        'rmse': rmse,
        'r2': r2  
    }

In [ ]:
def objective(trial):
    
    params = {
        'lr': trial.suggest_float('lr', 1e-5, 1e-3, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64]),
        'epochs': trial.suggest_int('epochs', 20, 100),
        'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
    }
    
    kf = KFold(n_splits=5)
    rmse_scores = []  
    r2_scores = []  
    

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
   
        model = train_model(X_train, y_train, params)
      
        metrics = evaluate_model(model, X_val, y_val)
        
  
        rmse_scores.append(metrics['rmse'])
        r2_scores.append(metrics['r2'])

    return np.mean(rmse_scores)


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)
best_params = study.best_params
print("Best hyperparameters:", best_params)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

final_model = train_model(X_train, y_train, best_params)

results = evaluate_model(final_model, X_test, y_test)

In [ ]:
print("\nFinal Evaluation Results:")
print(f"Mean Squared Error (MSE): {results['mse']:.4f}")
print(f"Root Mean Squared Error (RMSE): {results['rmse']:.4f}")
print(f"R-squared (R2): {results['r2']:.4f}")


In [ ]:

plt.figure(figsize=(10, 6))
sns.regplot(x=y_test, y=results['predictions'], scatter_kws={'alpha':0.3})
plt.xlabel('Actual Scores')
plt.ylabel('Predicted Scores')
plt.title('Actual vs Predicted Essay Scores')
plt.show()



In [ ]:
errors = y_test - results['predictions']
plt.figure(figsize=(10, 6))
sns.histplot(errors, bins=30, kde=True)
plt.xlabel('Prediction Error')
plt.title('Distribution of Prediction Errors')
plt.show()

---
